# Python для анализа данных

## Бот в Telegram

### Сбор статистики по Коронавирусу

*Ян Пиле*

### Есть сайт с информацией о распространении вируса в мире
Я предлагаю достать оттуда информацию в виде Pandas dataframe а затем по запросу пользователя в телеграм выгружать ее

Импортируем requests и запишем адрес сайта со статистикой по COVID в переменную

In [8]:
import requests
from bs4 import BeautifulSoup

После чего через get-запрос достанем текст страницы со статистикой и распарсим ее с помощью BeautifulSoup

In [9]:
url = 'https://www.worldometers.info/coronavirus/'
website = requests.get(url).text
soup = BeautifulSoup(website, 'lxml')
# soup

In [33]:
soup.find_all('table')[1]

<table class="table table-bordered table-hover main_table_countries" id="main_table_countries_yesterday" style="width:100%;margin-top: 0px !important;display:none;">
<thead>
<tr>
<th width="1%">#</th>
<th width="100">Country,<br/>Other</th>
<th width="20">Total<br/>Cases</th>
<th width="30">New<br/>Cases</th>
<th width="30">Total<br/>Deaths</th>
<th width="30">New<br/>Deaths</th>
<th width="30">Total<br/>Recovered</th>
<th width="30">New<br/>Recovered</th>
<th width="30">Active<br/>Cases</th>
<th width="30">Serious,<br/>Critical</th>
<th width="30">Tot Cases/<br/>1M pop</th>
<th width="30">Deaths/<br/>1M pop</th>
<th width="30">Total<br/>Tests</th>
<th width="30">Tests/<br/>
<nobr>1M pop</nobr>
</th>
<th width="30">Population</th>
<th style="display:none" width="30">Continent</th>
<th width="30">1 Case<br/>every X ppl</th><th width="30">1 Death<br/>every X ppl</th><th width="30">1 Test<br/>every X ppl</th>
<th width="30">New Cases/1M pop</th>
<th width="30">New Deaths/1M pop</th>
<th w

На странице две таблицы: одна отвечает за информацию на сегодняшний день (под номером 0), а вторая - за информацию на вчерашний день. Достанем обе таблицы (они лежат между тегами \<\table\>\)

In [10]:
table = soup.find_all('table')[0]
# table

In [31]:
len(soup.find_all('table'))

3

Разобьем таблицу на строки (они лежат между тегами tr)

In [11]:
rows = table.find_all('tr')
rows

[<tr>
 <th width="1%">#</th>
 <th width="100">Country,<br/>Other</th>
 <th width="20">Total<br/>Cases</th>
 <th width="30">New<br/>Cases</th>
 <th width="30">Total<br/>Deaths</th>
 <th width="30">New<br/>Deaths</th>
 <th width="30">Total<br/>Recovered</th>
 <th width="30">New<br/>Recovered</th>
 <th width="30">Active<br/>Cases</th>
 <th width="30">Serious,<br/>Critical</th>
 <th width="30">Tot Cases/<br/>1M pop</th>
 <th width="30">Deaths/<br/>1M pop</th>
 <th width="30">Total<br/>Tests</th>
 <th width="30">Tests/<br/>
 <nobr>1M pop</nobr>
 </th>
 <th width="30">Population</th>
 <th style="display:none" width="30">Continent</th>
 <th width="30">1 Case<br/>every X ppl</th><th width="30">1 Death<br/>every X ppl</th><th width="30">1 Test<br/>every X ppl</th>
 <th width="30">New Cases/1M pop</th>
 <th width="30">New Deaths/1M pop</th>
 <th width="30">Active Cases/1M pop</th>
 </tr>,
 <tr class="total_row_world row_continent" data-continent="North America" style="display: none">
 <td></td>


Сначала достанем заголовки (между тегами th). Для примера достанем заголовок первой колонки.

In [14]:
rows[0].find_all('th')[1].get_text().strip()

'Country,Other'

Теперь мы можем достать содержимое ячеек таблицы (между тегами td). Для примера достанем первую запись первого по счету столбца

In [33]:
rows[1].find_all('td')[1].get_text().strip()

'North America'

Протестируем сборку на примере одной колонки, а потом пробежимся по их списку в цикле

In [15]:
col1 = []

col1.append(rows[0].find_all('th')[1].get_text().strip()) # отдельно добавляем заголовок

for row in rows[1:]: # начинаем со второго ряда таблицы, потому что 0 уже обработали выше
    r = row.find_all('td') # находим все теги td для строки таблицы
    col1.append(r[1].get_text().strip()) # сохраняем данные в наш список

print(col1)

['Country,Other', 'North America', 'Asia', 'South America', 'Europe', 'Oceania', 'Africa', '', 'World', 'USA', 'India', 'Brazil', 'France', 'UK', 'Germany', 'Russia', 'Turkey', 'Italy', 'S. Korea', 'Spain', 'Argentina', 'Vietnam', 'Netherlands', 'Iran', 'Japan', 'Colombia', 'Indonesia', 'Poland', 'Mexico', 'Ukraine', 'Australia', 'Malaysia', 'Israel', 'Czechia', 'Belgium', 'Austria', 'South Africa', 'Philippines', 'Thailand', 'Peru', 'Portugal', 'Chile', 'Canada', 'Switzerland', 'Greece', 'Denmark', 'Romania', 'Sweden', 'Iraq', 'Serbia', 'Bangladesh', 'Hungary', 'Slovakia', 'Jordan', 'Georgia', 'Pakistan', 'Ireland', 'Norway', 'Kazakhstan', 'Morocco', 'Hong Kong', 'Bulgaria', 'Croatia', 'Lebanon', 'Cuba', 'Singapore', 'Tunisia', 'Lithuania', 'Nepal', 'Belarus', 'Slovenia', 'Bolivia', 'UAE', 'Uruguay', 'Finland', 'Ecuador', 'Costa Rica', 'Guatemala', 'Azerbaijan', 'Latvia', 'Panama', 'Saudi Arabia', 'Sri Lanka', 'Paraguay', 'Kuwait', 'Myanmar', 'New Zealand', 'Palestine', 'Dominican Rep

In [18]:
col1[:236][-1]

'China'

Чтобы еще раз проверить, запустим для другой колонки - TotalCases

In [19]:
col2 = []

col2.append(rows[0].find_all('th')[2].get_text().strip()) # отдельно добавляем заголовок

for row in rows[1:]: # начинаем со второго ряда таблицы, потому что 0 уже обработали выше
    r = row.find_all('td') # находим все теги td для строки таблицы
    col2.append(r[2].get_text().strip()) # сохраняем данные в наш список

print(col2)

['TotalCases', '96,355,732', '137,359,259', '55,984,522', '175,971,305', '5,204,198', '11,691,945', '721', '482,567,682', '81,621,888', '43,020,723', '29,842,418', '25,029,573', '20,691,123', '20,387,275', '17,783,843', '14,800,677', '14,396,283', '12,003,054', '11,451,676', '9,026,075', '9,011,473', '7,775,389', '7,154,003', '6,377,719', '6,083,643', '6,001,751', '5,945,594', '5,650,896', '4,952,913', '4,326,294', '4,138,867', '3,864,155', '3,790,322', '3,782,085', '3,747,582', '3,713,252', '3,677,376', '3,553,720', '3,545,403', '3,534,003', '3,455,060', '3,441,488', '3,437,733', '2,957,810', '2,899,802', '2,842,739', '2,481,736', '2,318,450', '1,970,827', '1,951,363', '1,844,581', '1,690,203', '1,689,314', '1,646,545', '1,524,086', '1,422,945', '1,397,638', '1,305,065', '1,162,974', '1,136,192', '1,133,464', '1,094,202', '1,090,782', '1,088,017', '1,072,005', '1,033,731', '1,016,117', '978,347', '959,802', '957,720', '901,367', '890,398', '884,081', '859,477', '857,299', '834,726', '

А теперь пройдемся циклом по 9 колонкам, сгружая данные в список списков

In [25]:
field_list = []
for i in range(0,10):
    col = []
    col.append(rows[0].find_all('th')[i+1].get_text().strip()) # отдельно добавляем заголовок
    for row in rows[1:]: # начинаем со второго ряда таблицы, потому что 0 уже обработали выше
        r = row.find_all('td') # находим все теги td для строки таблицы
        col.append(r[i+1].get_text().strip()) # сохраняем данные в наш список
    field_list.append(col)

А вот заголовок второго столбца

In [26]:
field_list[2][0]

'NewCases'

Остается превратить эту структуру в DataFrame. Для этого сначала превратим ее в словарь из списков, где ключами будут названия колонок.

In [27]:
d = dict()
for i in range(9):
    d[field_list[i][0]] = field_list[i][1:]
# d

А теперь сгружаем этот словарь в функцию pd.DataFrame и он превращается в искомый DataFrame

In [28]:
import pandas as pd 
df = pd.DataFrame(d)
df.head(10)

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical"
0,North America,"96,355,732","+1,753","1,438,291",+16,"77,519,718","+2,584","17,397,723","7,881"
1,Asia,"137,359,259","+275,646","1,398,479",+797,"117,735,279","+94,223","18,225,501","26,783"
2,South America,"55,984,522","+4,543","1,286,325",+65,"51,338,480","+31,079","3,359,717","11,671"
3,Europe,"175,971,305","+333,075","1,765,446",+877,"155,007,423","+545,322","19,198,436","10,704"
4,Oceania,"5,204,198","+60,050","8,720",+15,"4,522,125","+15,019","673,353",153
5,Africa,"11,691,945",+175,"252,445",+4,"10,892,465",+831,"547,035","1,412"
6,,721,,15,,706,,0,0
7,World,"482,567,682","+675,242","6,149,721","+1,774","417,016,196","+689,058","59,401,765","58,604"
8,USA,"81,621,888",,"1,003,467",,"64,457,026",,"16,161,395","2,257"
9,India,"43,020,723",,"521,066",,"42,483,829",,"15,828","8,944"


In [43]:
df.head(8)

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical"
0,North America,"17,154,525","+13,774","424,160",+808,"10,451,820","+14,223","6,278,545","30,894"
1,Asia,"17,278,985","+18,231","298,827",+249,"15,279,958","+12,559","1,700,200","28,023"
2,South America,"11,431,372",+139,"330,541",+5,"10,181,268",+375,"919,563","16,992"
3,Europe,"18,135,234","+51,275","417,772","+1,037","8,006,804","+44,890","9,710,658","27,647"
4,Africa,"2,248,159",,"53,368",,"1,916,443",,"278,348","2,559"
5,Oceania,"45,918",+16,"1,021",,"33,145",+15,"11,752",25
6,,721,,15,,706,,0,0
7,World,"66,294,914","+83,435","1,525,704","+2,099","45,870,144","+72,062","18,899,066","106,140"


Для удобства переименуем колонки с составными названиями.

In [29]:
df = df.rename(columns={'Country,Other': 'Country', 'Serious,Critical': 'SeriousCritical'})

In [30]:
df.head(8)

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,SeriousCritical
0,North America,"96,355,732","+1,753","1,438,291",+16,"77,519,718","+2,584","17,397,723","7,881"
1,Asia,"137,359,259","+275,646","1,398,479",+797,"117,735,279","+94,223","18,225,501","26,783"
2,South America,"55,984,522","+4,543","1,286,325",+65,"51,338,480","+31,079","3,359,717","11,671"
3,Europe,"175,971,305","+333,075","1,765,446",+877,"155,007,423","+545,322","19,198,436","10,704"
4,Oceania,"5,204,198","+60,050","8,720",+15,"4,522,125","+15,019","673,353",153
5,Africa,"11,691,945",+175,"252,445",+4,"10,892,465",+831,"547,035","1,412"
6,,721,,15,,706,,0,0
7,World,"482,567,682","+675,242","6,149,721","+1,774","417,016,196","+689,058","59,401,765","58,604"


А теперь сгрузим весь процесс, который мы проделали, в одну функцию. Ее аргумент tag = 0 - информация за сегодняшний день, a tag=1 - информация за вчерашний день.

In [34]:
def stat(tag = 0):
    url = 'https://www.worldometers.info/coronavirus/'
    website = requests.get(url).text
    soup = BeautifulSoup(website, 'lxml')
    table = soup.find_all('table')[tag]
    rows = table.find_all('tr')
    d = dict()
    for i in range(10):
        col = []
        key = rows[0].find_all('th')[i+1].get_text().strip()
        
        for row in rows[1:236]:
            r = row.find_all('td')
            col.append(r[i+1].get_text().strip())
        d[key] = col
    df = pd.DataFrame(d)
    df = df.rename(columns = {'Country,Other':'Country', 'Serious,Critical':'SeriousCritical'})
    return df

In [35]:
stat().head(10)

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,SeriousCritical,Tot Cases/1M pop
0,North America,"96,355,732","+1,753","1,438,291",+16,"77,519,718","+2,584","17,397,723","7,881",
1,Asia,"137,359,259","+275,646","1,398,479",+797,"117,735,279","+94,223","18,225,501","26,783",
2,South America,"55,984,522","+4,543","1,286,325",+65,"51,338,480","+31,079","3,359,717","11,671",
3,Europe,"175,991,237","+338,338","1,765,457",+888,"155,011,018","+548,917","19,214,762","10,704",
4,Oceania,"5,204,198","+60,050","8,720",+15,"4,522,125","+15,019","673,353",153,
5,Africa,"11,691,995",+225,"252,446",+5,"10,892,564",+930,"546,985","1,412",
6,,721,,15,,706,,0,0,
7,World,"482,587,664","+680,555","6,149,733","+1,786","417,019,890","+692,752","59,418,041","58,604","61,911"
8,USA,"81,621,888",,"1,003,467",,"64,457,026",,"16,161,395","2,257","244,111"
9,India,"43,020,723",,"521,066",,"42,483,829",,"15,828","8,944","30,653"


In [37]:
list(stat()['Country'][:8])

['North America',
 'Asia',
 'South America',
 'Europe',
 'Oceania',
 'Africa',
 '',
 'World']

В первых 8 строках таблицы в колонке Country лежат регионы + Мир. Одна строка пустая. Уберем ее с помощью спискового включения.

In [38]:
print(*list(i+'\n' for i in list(stat()['Country'][:8]) if i!=''))

North America
 Asia
 South America
 Europe
 Oceania
 Africa
 World



А с 8 по 220 строки лежат названия стран. Выведем ее с помощью спискового включения по 6 стран в линию (для этого вставим символ переноса строки)

In [39]:
x = stat()['Country'][8:236]
', '.join([e+'\n' if i%6 == 5 else e for i,e in enumerate(x)])

'USA, India, Brazil, France, UK, Germany\n, Russia, Turkey, Italy, S. Korea, Spain, Argentina\n, Vietnam, Netherlands, Iran, Japan, Colombia, Indonesia\n, Poland, Mexico, Ukraine, Australia, Malaysia, Israel\n, Czechia, Belgium, Austria, South Africa, Philippines, Thailand\n, Peru, Portugal, Chile, Canada, Switzerland, Greece\n, Denmark, Romania, Sweden, Iraq, Serbia, Bangladesh\n, Hungary, Slovakia, Jordan, Georgia, Pakistan, Ireland\n, Norway, Kazakhstan, Morocco, Hong Kong, Bulgaria, Croatia\n, Lebanon, Cuba, Singapore, Tunisia, Lithuania, Nepal\n, Belarus, Slovenia, Bolivia, UAE, Uruguay, Finland\n, Ecuador, Costa Rica, Guatemala, Azerbaijan, Latvia, Panama\n, Saudi Arabia, Sri Lanka, Paraguay, Kuwait, Myanmar, New Zealand\n, Palestine, Dominican Republic, Estonia, Bahrain, Venezuela, Moldova\n, Libya, Egypt, Ethiopia, Mongolia, Armenia, Honduras\n, Cyprus, Oman, Bosnia and Herzegovina, Qatar, Réunion, Kenya\n, Zambia, North Macedonia, Albania, Algeria, Botswana, Nigeria\n, Zimbabw

In [40]:
x = stat()['Country'][8:236]
import re
# print(x)
print(', '.join([e+'\n' if i%6 == 5 else e for i,e in enumerate(x)]).replace('\n,',',\n'))

USA, India, Brazil, France, UK, Germany,
 Russia, Turkey, Italy, S. Korea, Spain, Argentina,
 Vietnam, Netherlands, Iran, Japan, Colombia, Indonesia,
 Poland, Mexico, Ukraine, Australia, Malaysia, Israel,
 Czechia, Belgium, Austria, South Africa, Philippines, Thailand,
 Peru, Portugal, Chile, Canada, Switzerland, Greece,
 Denmark, Romania, Sweden, Iraq, Serbia, Bangladesh,
 Hungary, Slovakia, Jordan, Georgia, Pakistan, Ireland,
 Norway, Kazakhstan, Morocco, Hong Kong, Bulgaria, Croatia,
 Lebanon, Cuba, Singapore, Tunisia, Lithuania, Nepal,
 Belarus, Slovenia, Bolivia, UAE, Uruguay, Finland,
 Ecuador, Costa Rica, Guatemala, Azerbaijan, Latvia, Panama,
 Saudi Arabia, Sri Lanka, Paraguay, Kuwait, Myanmar, New Zealand,
 Palestine, Dominican Republic, Estonia, Bahrain, Venezuela, Moldova,
 Libya, Egypt, Ethiopia, Mongolia, Armenia, Honduras,
 Cyprus, Oman, Bosnia and Herzegovina, Qatar, Réunion, Kenya,
 Zambia, North Macedonia, Albania, Algeria, Botswana, Nigeria,
 Zimbabwe, Uzbekistan, Mon

Для вывода результатов в текстовом формате воспользуемся библиотекой tabulate (в таком формате мы будем отправлять результаты в telegram'е)

In [52]:
!pip install tabulate

In [43]:
from tabulate import tabulate

Например общее количество случаев заболевания коронавирусом в Великобритании вывести можно так:

In [44]:
df[df['Country']=='UK'][['Country','TotalCases']].reset_index(drop=True)

,Country,TotalCases
0,UK,"20,691,123"


In [45]:
x = df[df['Country']=='UK'][['Country','TotalCases']].reset_index(drop=True)
print(tabulate(x, headers = x.columns, tablefmt="grid"))

+----+-----------+--------------+
|    | Country   | TotalCases   |
+====+===========+==============+
|  0 | UK        | 20,691,123   |
+----+-----------+--------------+


In [ ]:
!pip install semidbm

In [1]:
import semidbm


In [2]:
db = semidbm.open('testdb', 'c')
db['foo'] = 'bar'
print(db['foo'].decode())
db.close()

bar


In [3]:
db = semidbm.open('testdb', 'c')
db['foo'].decode()

'bar'

In [50]:
db = semidbm.open('testdb', 'c')
del db['foo']

In [51]:
db['foo']

KeyError: b'foo'

In [52]:
db.close()